In [26]:
import pandas as pd
data_cardio = pd.read_csv("https://raw.githubusercontent.com/lopeslopesedu/padroes/main/cardio_train.csv", sep=";")
data_cardio.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [27]:
X, y = data_cardio.drop(['cardio'], axis=1, inplace=False), data_cardio['cardio']

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=31, shuffle=True)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
vt_class =  VotingClassifier([('KNN', KNeighborsClassifier()), ('TREE', DecisionTreeClassifier()), ('RF', RandomForestClassifier())], n_jobs=-1)
pipeline = Pipeline([('ensemble', vt_class)])
parametros = {'ensemble__KNN__n_neighbors':[7,14],'ensemble__RF__max_depth':[7,14],'ensemble__RF__criterion' :['gini', 'entropy']}
modelo_parametrizado = GridSearchCV(pipeline, param_grid=parametros, return_train_score = True, n_jobs=-1)
scores_vt = cross_validate(modelo_parametrizado, X_train, y_train, return_estimator=True)

In [ ]:
import numpy as np
scores_vt, np.mean(scores_vt['test_score'])

({'estimator': (GridSearchCV(cv=None, error_score=nan,
                estimator=Pipeline(memory=None,
                                   steps=[('ensemble',
                                           VotingClassifier(estimators=[('KNN',
                                                                         KNeighborsClassifier(algorithm='auto',
                                                                                              leaf_size=30,
                                                                                              metric='minkowski',
                                                                                              metric_params=None,
                                                                                              n_jobs=None,
                                                                                              n_neighbors=5,
                                                                                              p=2,

In [ ]:
classificador = scores_vt['estimator'][np.argmax(scores_vt['test_score'])].best_estimator_
ypred = classificador.predict(Xteste)
acertos = y_test == ypred
acertos, sum(acertos)/len(acertos)

(46730     True
 48393     True
 41416     True
 34506     True
 43725     True
          ...  
 33226     True
 23125    False
 51551     True
 54916     True
 67572    False
 Name: cardio, Length: 17500, dtype: bool, 0.6712)

Stacking

In [ ]:
import warnings
from xgboost import XGBClassifier
warnings.filterwarnings('ignore')
estimadores =  XGBClassifier(n_jobs=-1)
pipeline = Pipeline([('ensemble', estimadores)])
parametros = {'ensemble__use_label_encoder':[True, False]}
modelo_parametrizado = GridSearchCV(pipeline, param_grid=parametros, return_train_score = True, n_jobs=-1)
scores_stacking = cross_validate(modelo_parametrizado, X_train, y_train, return_estimator=True)
scores_stacking, np.mean(scores_stacking['test_score'])

({'estimator': (GridSearchCV(cv=None, error_score=nan,
                estimator=Pipeline(memory=None,
                                   steps=[('ensemble',
                                           XGBClassifier(base_score=0.5,
                                                         booster='gbtree',
                                                         colsample_bylevel=1,
                                                         colsample_bynode=1,
                                                         colsample_bytree=1,
                                                         gamma=0,
                                                         learning_rate=0.1,
                                                         max_delta_step=0,
                                                         max_depth=3,
                                                         min_child_weight=1,
                                                         missing=None,
                           

In [ ]:
classificador = scores_stacking['estimator'][np.argmax(scores_stacking['test_score'])].best_estimator_
ypred = classificador.predict(X_test)
acertos = y_test == ypred
acertos, sum(acertos)/len(acertos)

(46730     True
 48393     True
 41416     True
 34506    False
 43725     True
          ...  
 33226    False
 23125    False
 51551     True
 54916     True
 67572    False
 Name: cardio, Length: 17500, dtype: bool, 0.7371428571428571)